In [1]:
import pandas as pd
celiac_df=pd.read_csv("/Users/admin/Downloads/celiac_disease_lab_data.csv")
import seaborn as sns
sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt

In [2]:
celiac_df.describe()

,Age,IgA,IgG,IgM
count,2206.000000,2206.000000,2206.000000,2206.000000
mean,12.768812,1.427384,10.052901,1.236963
std,7.434250,1.110042,2.047683,0.447216
min,1.000000,0.340000,5.000000,0.500000
25%,8.000000,1.000000,8.700000,0.980000
50%,10.000000,1.100000,10.000000,1.100000
75%,15.000000,1.800000,12.000000,1.500000
max,35.000000,9.000000,15.300000,2.700000


In [4]:
# Filter the DataFrame for "yes" diagnosis
yes_diagnosis_df = celiac_df[celiac_df['Disease_Diagnose'] == 'yes']

# Generate descriptive statistics for "yes" diagnosis
yes_diagnosis_stats = yes_diagnosis_df.describe()

# Filter the DataFrame for "no" diagnosis
no_diagnosis_df = celiac_df[celiac_df['Disease_Diagnose'] == 'no']

# Generate descriptive statistics for "no" diagnosis
no_diagnosis_stats = no_diagnosis_df.describe()

# Print the statistics
print("Statistics for 'yes' diagnosis:")
print(yes_diagnosis_stats)

print("\nStatistics for 'no' diagnosis:")
print(no_diagnosis_stats)

Statistics for 'yes' diagnosis:
               Age          IgA          IgG          IgM
count  1843.000000  1843.000000  1843.000000  1843.000000
mean     13.234943     1.305708    10.157027     1.231812
std       7.856934     0.546986     1.982584     0.467183
min       1.000000     0.340000     5.000000     0.500000
25%       8.000000     1.000000     8.700000     0.910000
50%      10.000000     1.100000    10.000000     1.100000
75%      16.000000     1.800000    12.000000     1.500000
max      35.000000     2.600000    15.300000     2.700000

Statistics for 'no' diagnosis:
              Age         IgA         IgG         IgM
count  363.000000  363.000000  363.000000  363.000000
mean    10.402204    2.045152    9.524242    1.263113
std      3.976199    2.350532    2.281263    0.326851
min      3.000000    0.340000    5.000000    0.600000
25%      8.000000    1.000000    8.000000    1.000000
50%      9.000000    1.300000    9.400000    1.100000
75%     12.000000    1.900000   12.0